Separate Notebook due to slowness.

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, mean_squared_error,r2_score
from sklearn.metrics import confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV

In [3]:
LFBclass = pd.read_csv('../Assets/LFBClass.csv')

In [4]:
LFBclass['ob_time'] = pd.to_datetime(LFBclass.ob_time)
LFBclass.set_index('ob_time', inplace=True, drop=True)

<ipython-input-4-bbee4ccaf52c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LFBclass['lockdown'][i] = 1


In [5]:
LFBclass.head()

,HourOfCall,IncidentGroup,PropertyType,holiday,weekend,air_temperature,cld_ttl_amt_id,islight,max_gust_speed,prcp_amt,rltv_hum,stn_pres,visibility,wind_speed,wmo_hr_sun_dur,wind_dirc,lockdown
ob_time,,,,,,,,,,,,,,,,,
2016-01-01,0,False Alarm,House - single occupancy,1,0,3.8,0.0,0,14.0,0.0,89.8,1017.8,2000.0,6.0,0.0,13,0
2016-01-01,0,Special Service,River/canal,1,0,3.8,0.0,0,14.0,0.0,89.8,1017.8,2000.0,6.0,0.0,13,0
2016-01-01,0,Special Service,Purpose Built Flats/Maisonettes - 4 to 9 storeys,1,0,3.8,0.0,0,14.0,0.0,89.8,1017.8,2000.0,6.0,0.0,13,0
2016-01-01,0,Special Service,Purpose Built Flats/Maisonettes - 4 to 9 storeys,1,0,3.8,0.0,0,14.0,0.0,89.8,1017.8,2000.0,6.0,0.0,13,0
2016-01-01,0,False Alarm,Car,1,0,3.8,0.0,0,14.0,0.0,89.8,1017.8,2000.0,6.0,0.0,13,0


In [6]:
LFBclass.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 518239 entries, 2016-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   HourOfCall       518239 non-null  int64  
 1   IncidentGroup    518239 non-null  object 
 2   PropertyType     518239 non-null  object 
 3   holiday          518239 non-null  int64  
 4   weekend          518239 non-null  int64  
 5   air_temperature  518239 non-null  float64
 6   cld_ttl_amt_id   518239 non-null  float64
 7   islight          518239 non-null  int64  
 8   max_gust_speed   518239 non-null  float64
 9   prcp_amt         518239 non-null  float64
 10  rltv_hum         518239 non-null  float64
 11  stn_pres         518239 non-null  float64
 12  visibility       518239 non-null  float64
 13  wind_speed       518239 non-null  float64
 14  wmo_hr_sun_dur   518239 non-null  float64
 15  wind_dirc        518239 non-null  int64  
 16  lock

In [7]:
LFBclass.describe(include = 'all')

,HourOfCall,IncidentGroup,PropertyType,holiday,weekend,air_temperature,cld_ttl_amt_id,islight,max_gust_speed,prcp_amt,rltv_hum,stn_pres,visibility,wind_speed,wmo_hr_sun_dur,wind_dirc,lockdown
count,518239.000000,518239,518239,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000,518239.000000
unique,NaN,3,286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,False Alarm,Purpose Built Flats/Maisonettes - 4 to 9 storeys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,260239,65922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,13.469129,NaN,NaN,0.023451,0.288620,12.974926,5.047432,0.524721,17.153237,0.077282,72.482067,1012.740564,2753.068580,8.279057,0.154236,10.537368,0.054454
std,6.242777,NaN,NaN,0.151330,0.453121,6.549272,3.228197,0.499389,10.160868,0.395443,17.882359,10.962255,1414.094253,4.449349,0.322201,4.686315,0.226911
min,0.000000,NaN,NaN,0.000000,0.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,19.000000,967.800000,6.000000,0.000000,0.000000,1.000000,0.000000
25%,9.000000,NaN,NaN,0.000000,0.000000,8.100000,2.000000,0.000000,11.000000,0.000000,59.600000,1006.400000,1700.000000,5.000000,0.000000,7.000000,0.000000
50%,14.000000,NaN,NaN,0.000000,0.000000,12.400000,7.000000,1.000000,15.000000,0.000000,75.700000,1013.900000,2800.000000,8.000000,0.000000,12.000000,0.000000
75%,19.000000,NaN,NaN,0.000000,1.000000,17.600000,8.000000,1.000000,21.000000,0.000000,87.300000,1020.000000,4000.000000,11.000000,0.000000,14.000000,0.000000


In [8]:
y = LFBclass.pop('IncidentGroup')
y.value_counts(normalize = True)

False Alarm        0.502160
Special Service    0.313774
Fire               0.184066
Name: IncidentGroup, dtype: float64

In [9]:
dummify = ['wind_dirc','PropertyType', 'HourOfCall']
X = pd.get_dummies(LFBclass, columns=dummify, drop_first=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.9, stratify=y, random_state=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
from sklearn.svm import SVC
from sklearn import svm

In [12]:
clf = svm.SVC()
clf.fit(X_train, y_train)
print( 'Score ',clf.score(X_train, y_train))

Score  0.6635663701445305


In [13]:
print( 'Score ',clf.score(X_test, y_test))

Score  0.6579598470035333


In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, stratify=y, random_state=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = svm.SVC(verbose = True)
clf.fit(X_train, y_train)
print( 'Train Score ',clf.score(X_train, y_train))
print( 'Test Score ',clf.score(X_test, y_test))

[LibSVM]Train Score  0.6622574700666686
Test Score  0.6582881483482557
